In [1]:
!pip install argilla

  Installing build dependencies ... one
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached wrapt-1.17.2-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
Using cached wrapt-1.17.2-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (89 kB)
  Created wheel for standardwebhooks: filename=standardwebhooks-1.0.0-py3-none-any.whl size=3571 sha256=04ce57ba3164e63417275cbc5149566bb8c4100010a3421cc7a3d111469f86b6
  Stored in directory: /home/renwei/.cache/pip/wheels/40/c9/01/255026ce5c725bee8d87fcf67c5c2149847f31b5df60bb83db
Successfully built standardwebhooks


# Install and connect the Python SDK

https://huggingface.co/learn/nlp-course/chapter10/2?fw=pt

In [2]:
!pip install python-dotenv

In [3]:
# https://github.com/theskumar/python-dotenvabs
%load_ext dotenv
%dotenv ../.env

In [6]:
import os

ARGILLA_SPACE_API_KEY = os.getenv("ARGILLA_SPACE_API_KEY")

In [7]:
import argilla as rg

client = rg.Argilla(
    api_url="https://renwei2024-learn-argilla.hf.space/",
    api_key=ARGILLA_SPACE_API_KEY,
)

In [8]:
client.me

User(id=UUID('0f206382-caf1-446a-9248-ec74ee882b6f') inserted_at=datetime.datetime(2025, 1, 30, 1, 2, 17, 982720) updated_at=datetime.datetime(2025, 1, 30, 1, 2, 17, 982720) username='renwei2024' role=<Role.owner: 'owner'> first_name='renwei2024' last_name=None password=None)

# Load your dataset to Argilla

## Configure your dataset

In [10]:
from datasets import load_dataset

data = load_dataset("SetFit/ag_news", split="train")
data.features

train.jsonl:   0%|          | 0.00/33.8M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': Value(dtype='string', id=None),
 'label': Value(dtype='int64', id=None),
 'label_text': Value(dtype='string', id=None)}

In [11]:
data[0]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.",
 'label': 2,
 'label_text': 'Business'}

In [12]:
settings = rg.Settings(
    fields=[rg.TextField(name="text")],
    questions=[
        rg.LabelQuestion(
            name="label", title="Classify the text:", labels=data.unique("label_text")
        ),
        rg.SpanQuestion(
            name="entities",
            title="Highlight all the entities in the text:",
            labels=["PERSON", "ORG", "LOC", "EVENT"],
            field="text",
        ),
    ],
)

## Upload the dataset

In [13]:
dataset = rg.Dataset(name="ag_news", settings=settings)

dataset.create()

/home/renwei/anaconda3/envs/llm/lib/python3.12/site-packages/argilla/datasets/_resource.py:264: UserWarning: Workspace not provided. Using default workspace: argilla id: f228f253-a59f-44e8-956c-9ccad6db5863
  warnings.warn(f"Workspace not provided. Using default workspace: {workspace.name} id: {workspace.id}")


Dataset(id=UUID('64fab0ff-9014-4367-8bc7-a6d1b9427788') inserted_at=datetime.datetime(2025, 1, 30, 2, 23, 34, 63427) updated_at=datetime.datetime(2025, 1, 30, 2, 23, 34, 596754) name='ag_news' status='ready' guidelines=None allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('f228f253-a59f-44e8-956c-9ccad6db5863') last_activity_at=datetime.datetime(2025, 1, 30, 2, 23, 34, 596754))

In [14]:
dataset.records.log(data, mapping={"label_text": "label"})

/home/renwei/anaconda3/envs/llm/lib/python3.12/site-packages/argilla/records/_io/_datasets.py:265: UserWarning: Record id column not found in Hugging Face dataset. Using row index and split for record ids.
  warnings.warn(


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

/home/renwei/anaconda3/envs/llm/lib/python3.12/site-packages/argilla/records/_mapping/_mapper.py:89: UserWarning: Keys ['label_text'] in data are not present in the mapping and will be ignored.
  warnings.warn(f"Keys {unknown_keys} in data are not present in the mapping and will be ignored.")
Sending records...: 469batch [16:25,  2.10s/batch]                                                                      


DatasetRecords(Dataset(id=UUID('64fab0ff-9014-4367-8bc7-a6d1b9427788') inserted_at=datetime.datetime(2025, 1, 30, 2, 23, 34, 63427) updated_at=datetime.datetime(2025, 1, 30, 2, 23, 34, 596754) name='ag_news' status='ready' guidelines=None allow_extra_metadata=False distribution=OverlapTaskDistributionModel(strategy='overlap', min_submitted=1) workspace_id=UUID('f228f253-a59f-44e8-956c-9ccad6db5863') last_activity_at=datetime.datetime(2025, 1, 30, 2, 23, 34, 596754)))